<a href="https://colab.research.google.com/github/dannynacker/strobe_entrainment_periodicity_MSc/blob/main/aperiodic_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import numpy as np

def poisson_flash_times(frequency, duration, duty_cycle, min_interval=0.025, max_interval_factor=3.0):
    """Generates Poisson-distributed flash onset times ensuring correct flash count and duty cycle adjustments."""
    cycle_length = 1.0 / frequency
    expected_flashes = int(duration * frequency)
    times = []
    time = 0

    while len(times) < expected_flashes and time < duration:
        interval = np.random.exponential(cycle_length * 0.8)
        interval = min(interval, cycle_length * max_interval_factor)
        interval = max(interval, min_interval)

        on_duration = interval * (duty_cycle / 100)
        time += interval

        if time < duration:
            times.append((time, on_duration))

    return times

def calculate_effective_frequency(signal, duration):
    """Calculate effective frequency from generated flash times."""
    return len(signal) / duration

def optimize_initial_frequency(target_Fe, duration, duty_cycle, tolerance=0.01, max_iterations=10000, max_freq=200.0):
    """Finds the optimal initial frequency that produces the desired effective frequency, clamping within range."""
    F = min(target_Fe * 1.2, max_freq)  # Start slightly higher
    iteration = 0

    while iteration < max_iterations:
        flash_times = poisson_flash_times(F, duration, duty_cycle)
        Fe = calculate_effective_frequency(flash_times, duration)

        if abs(Fe - target_Fe) <= tolerance:
            return min(F, max_freq), Fe, flash_times

        F = min(max(0.1, F + 0.15 if Fe < target_Fe else F - 0.15), max_freq)
        iteration += 1

    print("⚠️ Warning: Max iterations reached without finding precise Fe match.")
    return F, Fe, flash_times

def generate_strobe_sequence():
    """Generates a strobe sequence file with proper STP formatting for all four oscillators in a single line."""
    stimulation = input("Choose stimulation type (Periodic/Aperiodic): ").strip()
    duration = float(input("Enter total sequence duration (seconds): "))
    start_Fe = float(input("Enter start effective frequency (Hz): "))
    end_Fe = float(input("Enter end effective frequency (Hz): "))
    start_l = int(input("Enter start luminance (0-100): "))
    end_l = int(input("Enter end luminance (0-100): "))
    start_d = int(input("Enter start duty cycle (1-99): "))
    end_d = int(input("Enter end duty cycle (1-99): "))
    wave_type = input("Enter wave type (Square/Sine): ").strip()
    led_config = [input(f"LED Set {i+1} (1=On, 0=Off): ") for i in range(4)]

    output = [f'TIM"00:00:{int(duration // 60):02}:{duration % 60:04.1f}"', f'DUR"{duration:.1f}"']

    num_steps = int(duration * (start_Fe + end_Fe) / 2)
    min_interval = 0.025

    if stimulation == "Aperiodic":
        F, Fe, flash_times = optimize_initial_frequency(start_Fe, duration, start_d)
    else:
        flash_times = [(i * (1.0 / start_Fe), (start_d / 100) * (1.0 / start_Fe)) for i in range(num_steps)]
        Fe = start_Fe

    previous_onset = 0
    step_lines = []

    for onset, on_duration in flash_times:
        step_duration = max(onset - previous_onset, min_interval)
        previous_onset = onset + on_duration

        step_freq = start_Fe if onset < duration / 2 else end_Fe
        step_duty = start_d if onset < duration / 2 else end_d
        step_luminance = start_l if onset < duration / 2 else end_l

        # **CORRECTED** STP formatting (all four oscillators in one line)
        stp_line = f'STP"{step_duration:.3f},1,{step_freq:.2f},{end_Fe:.2f},{step_duty},{end_d},' \
                   f'{led_config[0]},{led_config[1]},{led_config[2]},{led_config[3]},{step_luminance},{end_l}, ' \
                   f'{step_duration:.3f},1,{step_freq:.2f},{end_Fe:.2f},{step_duty},{end_d},' \
                   f'{led_config[0]},{led_config[1]},{led_config[2]},{led_config[3]},{step_luminance},{end_l}, ' \
                   f'{step_duration:.3f},1,{step_freq:.2f},{end_Fe:.2f},{step_duty},{end_d},' \
                   f'{led_config[0]},{led_config[1]},{led_config[2]},{led_config[3]},{step_luminance},{end_l}, ' \
                   f'{step_duration:.3f},1,{step_freq:.2f},{end_Fe:.2f},{step_duty},{end_d},' \
                   f'{led_config[0]},{led_config[1]},{led_config[2]},{led_config[3]},{step_luminance},{end_l}"'

        step_lines.append(stp_line)

    output.extend(step_lines)

    file_name = "strobe_periodic.txt" if stimulation == "Periodic" else "strobe_aperiodic.txt"

    with open(file_name, "w") as f:
        f.write("\n".join(output))

    print(f"✅ Generated {file_name} with correct formatting.")

# Run the generator
generate_strobe_sequence()

Choose stimulation type (Periodic/Aperiodic): Periodic
Enter total sequence duration (seconds): 5
Enter start effective frequency (Hz): 10
Enter end effective frequency (Hz): 10
Enter start luminance (0-100): 50
Enter end luminance (0-100): 50
Enter start duty cycle (1-99): 50
Enter end duty cycle (1-99): 50
Enter wave type (Square/Sine): Square
LED Set 1 (1=On, 0=Off): 1
LED Set 2 (1=On, 0=Off): 1
LED Set 3 (1=On, 0=Off): 1
LED Set 4 (1=On, 0=Off): 1
✅ Generated strobe_periodic.txt with correct formatting.
